#importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Inserting Data to workspace

In [ ]:
Trainig_Set_Path = '/content/Train.csv'
Trainig_Set_Data = pd.read_csv(Trainig_Set_Path)
Trainig_Set_Data
# Trainig_Set_Data.describe()
# Trainig_Set_Data.head()

#Structure of DataSet


In [ ]:
Trainig_Set_Data.info()

In [ ]:
Trainig_Set_Data.describe()

In [ ]:
Trainig_Set_Data.Item_Fat_Content.unique()

There is Missing data in Outlet_Size (s-m-high) and Item_Weight (number)
In Item_Fat_Content : LF should be Low Fat & reg should be Regular


In [ ]:
Trainig_Set_Data['Item_Fat_Content'] = Trainig_Set_Data['Item_Fat_Content'].replace('LF', 'Low Fat')
Trainig_Set_Data['Item_Fat_Content'] = Trainig_Set_Data['Item_Fat_Content'].replace('low fat', 'Low Fat')
Trainig_Set_Data['Item_Fat_Content'] = Trainig_Set_Data['Item_Fat_Content'].replace('reg', 'Regular')
Trainig_Set_Data['Item_Fat_Content'][85]

In [ ]:
Trainig_Set_Data.Item_Fat_Content.unique()

Replacing Done

In [ ]:
Trainig_Set_Data[pd.isnull(Trainig_Set_Data.Item_Weight)]

In [ ]:
Trainig_Set_Data.Item_Weight.info()

In [ ]:
# Trainig_Set_Data.Item_Weight.dtype
print(Trainig_Set_Data.dtypes)

#Filling all empty cells in Item_Weight col with specific value

In [ ]:
# Iterate over each row in the DataFrame
for index, row in Trainig_Set_Data.iterrows():
    # Check if Item_Weight value is NaN
    if pd.isnull(row['Item_Weight']):
        # Get the Item_Identifier name
        name = row['Item_Identifier']

        # Search for the name in Item_Identifier column
        mask = Trainig_Set_Data['Item_Identifier'] == name

        # Check if a non-NaN value exists in Item_Weight column for the given name
        if Trainig_Set_Data.loc[mask, 'Item_Weight'].notnull().any():
            # Get the non-NaN value from Item_Weight column
            value = Trainig_Set_Data.loc[mask, 'Item_Weight'].dropna().iloc[0]

            # Check if the value is not NaN in the same row
            if pd.isnull(Trainig_Set_Data.loc[index, 'Item_Weight']):
                # Replace NaN with the retrieved value
                Trainig_Set_Data.loc[index, 'Item_Weight'] = value

In [ ]:
Trainig_Set_Data[pd.isnull(Trainig_Set_Data.Item_Weight)]

Removing rows with Item_Identifier name with no Item_Weight Value in all dataset

In [ ]:
Trainig_Set_Data.dropna(subset=['Item_Weight'], inplace=True)

In [ ]:
Trainig_Set_Data[pd.isnull(Trainig_Set_Data.Item_Weight)]

#Filling all empty cells in Outlet_Size col with specific value

In [ ]:
# Iterate over each row in the DataFrame
for index, row in Trainig_Set_Data.iterrows():
    # Check if Outlet_Size value is NaN
    if pd.isnull(row['Outlet_Size']):
        # Get the Item_Identifier name
        name = row['Item_Identifier']

        # Search for the name in Item_Identifier column
        mask = Trainig_Set_Data['Item_Identifier'] == name

        # Check if a non-NaN value exists in Outlet_Size column for the given name
        if Trainig_Set_Data.loc[mask, 'Outlet_Size'].notnull().any():
            # Get the non-NaN value from Outlet_Size column
            value = Trainig_Set_Data.loc[mask, 'Outlet_Size'].dropna().iloc[0]

            # Check if the value is not NaN in the same row
            if pd.isnull(Trainig_Set_Data.loc[index, 'Outlet_Size']):
                # Replace NaN with the retrieved value
                Trainig_Set_Data.loc[index, 'Outlet_Size'] = value

In [ ]:
Trainig_Set_Data['Outlet_Size'][3]

'Small'

In [ ]:
Trainig_Set_Data[pd.isnull(Trainig_Set_Data.Outlet_Size)]

Removing rows with Item_Identifier name with no Outlet_Size Value in all dataset

In [ ]:
Trainig_Set_Data.dropna(subset=['Outlet_Size'], inplace=True)

In [ ]:
Trainig_Set_Data.head()

Done Cleaning Outlet_Size col

In [ ]:
Trainig_Set_Data.corr()

Item_MRP is the most effective feature on Item_Outlet_Sales (corr = 0.567702)

#Remove outliner values

In [ ]:
numeric_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year', 'Item_Outlet_Sales']

for column in numeric_columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=Trainig_Set_Data[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

In [ ]:
summary_stats = Trainig_Set_Data[numeric_columns].describe()
print(summary_stats)

In [ ]:
# Handling outliers (Approach: removing outliers)
def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Removing outliers
Trainig_Set_Data = remove_outliers(Trainig_Set_Data, 'Item_Outlet_Sales')
Trainig_Set_Data = remove_outliers(Trainig_Set_Data, 'Item_Visibility')

In [ ]:
updated_summary_stats = Trainig_Set_Data[numeric_columns].describe()
print(updated_summary_stats)

In [ ]:
Trainig_Set_Data.boxplot(column=["Item_Outlet_Sales"])

In [ ]:
Trainig_Set_Data.boxplot(column=["Item_Visibility"])

Done removing outliners

Checking for Duplicates

In [ ]:
duplicate = Trainig_Set_Data.duplicated()
print(duplicate.sum())

No Duplications found

#Downloading Updated DataSet

In [ ]:
# from google.colab import files

# # Save the updated DataFrame to a new CSV file
# Trainig_Set_Data.to_csv('updated_dataset.csv', index=False)

# # Download the CSV file
# files.download('updated_dataset.csv')

#Plotting Graphs

In [ ]:
# Visualize categorical variables
categorical_columns = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

for column in categorical_columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(data=Trainig_Set_Data, x=column)
    plt.title(f'Countplot of {column}')
    plt.xticks(rotation=45)
    plt.show()

# Visualize numerical variables
numeric_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales']

for column in numeric_columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(data=Trainig_Set_Data, x=column, kde=True)
    plt.title(f'Histogram of {column}')
    plt.show()

# Explore relationship between 'Item_MRP' and 'Item_Outlet_Sales'
plt.figure(figsize=(6, 4))
sns.scatterplot(data=Trainig_Set_Data, x='Item_MRP', y='Item_Outlet_Sales')
plt.title('Scatter plot: Item_MRP vs. Item_Outlet_Sales')
plt.show()

# Visualize correlations between numerical variables
numeric_data = Trainig_Set_Data[numeric_columns]
correlation_matrix = numeric_data.corr()

plt.figure(figsize=(6, 4))
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Applying Simple Linear Regression for All columns and Item_Outlet_Sales

#Encoding

Applying Label Encoding to Item_Fat_Content

In [ ]:
Trainig_Set_Data['Item_Fat_Content'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Trainig_Set_Data['Item_Fat_Content']= label_encoder.fit_transform(Trainig_Set_Data['Item_Fat_Content'])

Trainig_Set_Data['Item_Fat_Content'].unique()

Low Fat == 0 , Regular == 1

In [ ]:
Trainig_Set_Data['Item_Type'].unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

Applying OneHot Encoding to Item_Type after scailing

In [ ]:
onehot_encoder = OneHotEncoder(drop='first', sparse=False)
onehot_encoded = onehot_encoder.fit_transform(Trainig_Set_Data[['Item_Type']])
data_encoded = pd.concat([Trainig_Set_Data, pd.DataFrame(onehot_encoded)], axis=1)

In [ ]:
scaler = StandardScaler()
data_encoded[['Item_Weight', 'Item_Visibility', 'Item_MRP']] = scaler.fit_transform(data_encoded[['Item_Weight', 'Item_Visibility', 'Item_MRP']])

In [ ]:
Trainig_Set_Data.head()

In [ ]:
# Drop the original categorical feature 'Item_Type' after encoding
data_encoded.drop('Item_Type', axis=1, inplace=True)

Applying Label Encoding to Outlet_Size

In [ ]:
Trainig_Set_Data['Outlet_Size'].unique()

Replacing High with Large in Outlet_Size

In [ ]:
Trainig_Set_Data['Outlet_Size'] = Trainig_Set_Data['Outlet_Size'].replace('High', 'Large')

Encoding Outlet_Size

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Trainig_Set_Data['Outlet_Size']= label_encoder.fit_transform(Trainig_Set_Data['Outlet_Size'])

Trainig_Set_Data['Outlet_Size'].unique()

Encoding Outlet_Location_Type

In [ ]:
Trainig_Set_Data['Outlet_Location_Type'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Trainig_Set_Data['Outlet_Location_Type']= label_encoder.fit_transform(Trainig_Set_Data['Outlet_Location_Type'])

Trainig_Set_Data['Outlet_Location_Type'].unique()

Encoding Outlet_Type

In [ ]:
Trainig_Set_Data['Outlet_Type'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Trainig_Set_Data['Outlet_Type']= label_encoder.fit_transform(Trainig_Set_Data['Outlet_Type'])

Trainig_Set_Data['Outlet_Type'].unique()

Encoding Outlet_Identifier

In [ ]:
Trainig_Set_Data['Outlet_Identifier'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Trainig_Set_Data['Outlet_Identifier']= label_encoder.fit_transform(Trainig_Set_Data['Outlet_Identifier'])

Trainig_Set_Data['Outlet_Identifier'].unique()

In [ ]:
Trainig_Set_Data.head()

#Applying linear Regression to dataset

In [ ]:
# # Select the independent variables (features) and dependent variable (target)
# X = Trainig_Set_Data[['Item_Weight', 'Item_Visibility', 'Item_MRP']]
# y = Trainig_Set_Data['Item_Outlet_Sales']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# # Initialize the linear regression model
# model = LinearRegression()

# # Fit the model to the training data
# model.fit(X_train, y_train)

# # Predict on the test data
# y_pred = model.predict(X_test)

# # Evaluate the model's performance
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# # Print the evaluation metrics
# print("Mean Squared Error:", mse)
# print("R-squared Score:", r2)

# # Interpret the coefficients
# coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
# print(coefficients)

With only 'Item_Weight', 'Item_Visibility', 'Item_MRP' error is 1439081
and with all col error is 1438885.0179770319
so 'Item_Fat_Content' and 'Item_Type' not affecting error so much

So i can remove these 2 cols to simplify dataset


#Trying models

In [ ]:
# Select the independent variables (features) and dependent variable (target)
X = Trainig_Set_Data[['Item_MRP']]
y = Trainig_Set_Data['Item_Outlet_Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)


Linear Regression is the most accurate model

In [ ]:
# Initialize and train the models
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Predict on the test data
linear_regression_preds = linear_regression.predict(X_test)

# Evaluate the models' performance
linear_regression_mae = mean_absolute_error(y_test, linear_regression_preds)
linear_regression_r2 = r2_score(y_test, linear_regression_preds)

# Print the evaluation metrics
print("Linear Regression:")
print("Mean Absolute Error:", linear_regression_mae)
print("R-squared Score:", linear_regression_r2)
print()


In [ ]:
ridge_regression = Ridge(solver='cholesky')
ridge_regression.fit(X_train, y_train)
# Predict on the test data
ridge_regression_preds = ridge_regression.predict(X_test)

ridge_regression_mae = mean_absolute_error(y_test, ridge_regression_preds)
ridge_regression_r2 = r2_score(y_test, ridge_regression_preds)

print("Ridge Regression:")
print("Mean Absolute Error:", ridge_regression_mae)
print("R-squared Score:", ridge_regression_r2)
print()

In [ ]:
lasso_regression = Lasso()
lasso_regression.fit(X_train, y_train)
# Predict on the test data
lasso_regression_preds = lasso_regression.predict(X_test)

lasso_regression_mae = mean_absolute_error(y_test, lasso_regression_preds)
lasso_regression_r2 = r2_score(y_test, lasso_regression_preds)

print("Lasso Regression:")
print("Mean Absolute Error:", lasso_regression_mae)
print("R-squared Score:", lasso_regression_r2)
print()

In [ ]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)
# Predict on the test data
random_forest_preds = random_forest.predict(X_test)

random_forest_mae = mean_absolute_error(y_test, random_forest_preds)
random_forest_r2 = r2_score(y_test, random_forest_preds)

print("Random Forest:")
print("Mean Absolute Error:", random_forest_mae)
print("R-squared Score:", random_forest_r2)
print()

In [ ]:
xgboost = XGBRegressor()
xgboost.fit(X_train, y_train)
# Predict on the test data
xgboost_preds = xgboost.predict(X_test)

xgboost_mae = mean_absolute_error(y_test, xgboost_preds)
xgboost_r2 = r2_score(y_test, xgboost_preds)

print("XGBoost:")
print("Mean Absolute Error:", xgboost_mae)
print("R-squared Score:", xgboost_r2)


#Try predicting Item_outlet_Sales in test dataset

In [ ]:
Test_Set_Path = '/content/Test.csv'
Test_Set_Data = pd.read_csv(Test_Set_Path)
Test_Set_Data
# Trainig_Set_Data.describe()
# Trainig_Set_Data.head()

In [ ]:
Test_Set_Data.Item_Fat_Content.unique()

In [ ]:
Test_Set_Data['Item_Fat_Content'] = Test_Set_Data['Item_Fat_Content'].replace('LF', 'Low Fat')
Test_Set_Data['Item_Fat_Content'] = Test_Set_Data['Item_Fat_Content'].replace('low fat', 'Low Fat')
Test_Set_Data['Item_Fat_Content'] = Test_Set_Data['Item_Fat_Content'].replace('reg', 'Regular')
Test_Set_Data['Item_Fat_Content'][85]

'Regular'

In [ ]:
# Iterate over each row in the DataFrame
for index, row in Test_Set_Data.iterrows():
    # Check if Item_Weight value is NaN
    if pd.isnull(row['Item_Weight']):
        # Get the Item_Identifier name
        name = row['Item_Identifier']

        # Search for the name in Item_Identifier column
        mask = Test_Set_Data['Item_Identifier'] == name

        # Check if a non-NaN value exists in Item_Weight column for the given name
        if Test_Set_Data.loc[mask, 'Item_Weight'].notnull().any():
            # Get the non-NaN value from Item_Weight column
            value = Test_Set_Data.loc[mask, 'Item_Weight'].dropna().iloc[0]

            # Check if the value is not NaN in the same row
            if pd.isnull(Test_Set_Data.loc[index, 'Item_Weight']):
                # Replace NaN with the retrieved value
                Test_Set_Data.loc[index, 'Item_Weight'] = value

In [ ]:
Test_Set_Data[pd.isnull(Test_Set_Data.Item_Weight)]

In [ ]:
Test_Set_Data.dropna(subset=['Item_Weight'], inplace=True)

In [ ]:
Test_Set_Data[pd.isnull(Test_Set_Data.Item_Weight)]

In [ ]:
# Iterate over each row in the DataFrame
for index, row in Test_Set_Data.iterrows():
    # Check if Outlet_Size value is NaN
    if pd.isnull(row['Outlet_Size']):
        # Get the Item_Identifier name
        name = row['Item_Identifier']

        # Search for the name in Item_Identifier column
        mask = Test_Set_Data['Item_Identifier'] == name

        # Check if a non-NaN value exists in Outlet_Size column for the given name
        if Test_Set_Data.loc[mask, 'Outlet_Size'].notnull().any():
            # Get the non-NaN value from Outlet_Size column
            value = Test_Set_Data.loc[mask, 'Outlet_Size'].dropna().iloc[0]

            # Check if the value is not NaN in the same row
            if pd.isnull(Test_Set_Data.loc[index, 'Outlet_Size']):
                # Replace NaN with the retrieved value
                Test_Set_Data.loc[index, 'Outlet_Size'] = value

In [ ]:
Test_Set_Data[pd.isnull(Test_Set_Data.Outlet_Size)]

In [ ]:
Test_Set_Data.dropna(subset=['Outlet_Size'], inplace=True)

In [ ]:
Test_Set_Data.head()

In [ ]:
Test_Set_Data.corr()

In [ ]:
numeric_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']

for column in numeric_columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=Test_Set_Data[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

In [ ]:
summary_stats = Test_Set_Data[numeric_columns].describe()
print(summary_stats)

In [ ]:
# Handling outliers (Approach: removing outliers)
def remove_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Removing outliers
# Trainig_Set_Data = remove_outliers(Trainig_Set_Data, 'Item_Outlet_Sales')
Test_Set_Data = remove_outliers(Test_Set_Data, 'Item_Visibility')

In [ ]:
updated_summary_stats = Test_Set_Data[numeric_columns].describe()
print(updated_summary_stats)

In [ ]:
Test_Set_Data.boxplot(column=["Item_Visibility"])

In [ ]:
duplicate = Test_Set_Data.duplicated()
print(duplicate.sum())

In [ ]:
Test_Set_Data['Item_Fat_Content'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Test_Set_Data['Item_Fat_Content']= label_encoder.fit_transform(Test_Set_Data['Item_Fat_Content'])

Test_Set_Data['Item_Fat_Content'].unique()

In [ ]:
Test_Set_Data['Item_Type'].unique()

In [ ]:
onehot_encoder = OneHotEncoder(drop='first', sparse=False)
onehot_encoded = onehot_encoder.fit_transform(Test_Set_Data[['Item_Type']])
data_encoded = pd.concat([Test_Set_Data, pd.DataFrame(onehot_encoded)], axis=1)

In [ ]:
scaler = StandardScaler()
data_encoded[['Item_Weight', 'Item_Visibility', 'Item_MRP']] = scaler.fit_transform(data_encoded[['Item_Weight', 'Item_Visibility', 'Item_MRP']])

In [ ]:
# Drop the original categorical feature 'Item_Type' after encoding
data_encoded.drop('Item_Type', axis=1, inplace=True)

In [ ]:
Test_Set_Data['Outlet_Size'] = Test_Set_Data['Outlet_Size'].replace('High', 'Large')

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Test_Set_Data['Outlet_Size']= label_encoder.fit_transform(Test_Set_Data['Outlet_Size'])

Test_Set_Data['Outlet_Size'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Test_Set_Data['Outlet_Location_Type']= label_encoder.fit_transform(Test_Set_Data['Outlet_Location_Type'])

Test_Set_Data['Outlet_Location_Type'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Test_Set_Data['Outlet_Type']= label_encoder.fit_transform(Test_Set_Data['Outlet_Type'])

Test_Set_Data['Outlet_Type'].unique()

In [ ]:
# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
Test_Set_Data['Outlet_Identifier']= label_encoder.fit_transform(Test_Set_Data['Outlet_Identifier'])

Test_Set_Data['Outlet_Identifier'].unique()

array([9, 2, 0, 5, 8, 3, 7, 4, 1, 6])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [ ]:
# Select the independent variables/features from the new dataset
X_new = Test_Set_Data[['Item_MRP']]

# Use the trained XGBoost model to make predictions on the new dataset
predictions = xgboost.predict(X_new)

# Add the predicted values to the new dataset
Test_Set_Data['Item_Outlet_Sales_Predicted'] = predictions

# Print the new dataset with the predicted values
print(Test_Set_Data)

In [ ]:
Test_Set_Data['Item_Outlet_Sales'] = predictions

In [ ]:
Test_Set_Data.to_csv('new_dataset_with_predictions.csv', index=False)

# Download the new dataset with the added column
from google.colab import files
files.download('new_dataset_with_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>